In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:

pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [6]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [7]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [8]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_conc(artists):
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_conc = unique_artists / total_appearances
    return artist_conc

In [9]:
MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = []
    dates = []
    tracks = sp.tracks(uris)['tracks']
    for track in tracks:
        if track is None:
            print(track)
        else:
            for artist in track['artists']:
                artists.append(artist['name'])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates.append(ms_date / MS_IN_DAY)
            
    return artists, dates

In [10]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(tracks):
    tracks = pd.DataFrame(tracks)
    playlist_uris = tracks['track_uri']

    remainder = len(tracks) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(tracks):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    remainder = len(tracks) % 50
    artists, dates = [], []
    if remainder > 0:
        artists, dates = get_artists_and_dates(playlist_uris[:remainder])
    while remainder < len(tracks):
        next_artists, next_dates = get_artists_and_dates(playlist_uris[remainder:remainder+50])
        artists += next_artists
        dates += next_dates
        remainder += 50
    return all_features, artists, dates

In [11]:
# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features_v2(tracks):
    
    all_features, artists, dates = get_playlist_audio_features_batch(tracks)
    all_features = pd.DataFrame(all_features)

    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    aggregate_features['date'] = np.mean(dates)
    aggregate_features['date_var'] = np.var(dates)
    aggregate_features['artist_conc'] = get_artist_conc(artists)
    return aggregate_features

In [12]:
# build the aggregate dataset!
def build_dataset_v2(playlists):
    aggregate_dataset = []
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features_v2(playlist['tracks']))
    return aggregate_dataset

In [13]:
dataset = pd.DataFrame(build_dataset_v2(data['playlists'][:10]))
dataset.to_csv(r'csvdata_10.csv')

In [ ]:
print(dataset[3:6])

In [17]:
times = [1, 10, 100, 1000]
for i in times:
    t = timeit.Timer('build_dataset_v2(data[\'playlists\'][:i])', 'from __main__ import build_dataset_v2, data, i')
    print('Time: ' + str(t.timeit(1)))

Time: 0.3579309999997804
Time: 3.1406698749999578
Time: 46.32780729199999
None
None
Time: 438.3731665
